<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/06_Tablacategorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
ruta_historical ='/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx'

os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

# === Cargar histórico ===
if os.path.exists(ruta_historical):
    df = pd.read_parquet(ruta_historical)
    print(f"📊 Histórico cargado con {len(df)} registros.")
else:
    print("⚠️ No se encontró el archivo histórico.")
    exit()

📊 Histórico cargado con 9992 registros.


# Antiguo

In [ ]:
# ANTIGUO

# === Variables de cada tipo ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'S1', 'S2', 'S3', 'S4', 'S5M', 'C1M', 'C2M', 'T1M', 'T2M', 'T3']
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Lista de dimensiones ===
dimensiones = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
tablas_por_dimension = []

# === Función para generar análisis por dimensión ===
for dim in dimensiones:
    errores = df.groupby(['FECHA_EVALUACIÓN', dim])[columnas_eval_1].apply(lambda x: (x == False).sum()).reset_index()
    llamadas = df.groupby(['FECHA_EVALUACIÓN', dim]).size().reset_index(name='NUM_LLAMADAS')
    resultado = pd.merge(errores, llamadas, on=['FECHA_EVALUACIÓN', dim])

    resultado = resultado.melt(
        id_vars=['FECHA_EVALUACIÓN', dim, 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    resultado['%_CUMPLIMIENTO'] = ((1 - resultado['N_FALSOS'] / resultado['NUM_LLAMADAS']) * 100).round(2)
    resultado['PONDERACION'] = resultado['VARIABLE'].map(ponderacion_dict)

    resultado['DIMENSIÓN'] = dim.capitalize()
    resultado['VALOR_DIMENSIÓN'] = resultado[dim]
    resultado.drop(columns=[dim], inplace=True)

    tablas_por_dimension.append(resultado)

# === Concatenar todas las tablas por dimensión en una tabla general ===
tabla_general = pd.concat(tablas_por_dimension, ignore_index=True)

# === Cálculo %_CUMPLIMIENTO_P, C, S, T ===
def calcular_cumplimiento_grupo(df, grupo, divisor):
    variables_grupo = {
        'P': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6'],
        'S': ['S1', 'S2', 'S3', 'S4', 'S5M'],
        'C': ['C1M', 'C2M'],
        'T': ['T1M', 'T2M', 'T3']
    }
    temp = df[df['VARIABLE'].isin(variables_grupo[grupo])].copy()
    temp['PON_CUMP'] = temp['%_CUMPLIMIENTO'] * temp['PONDERACION']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum().reset_index()
    )
    resultado[f'%_CUMPLIMIENTO_{grupo}'] = (resultado['PON_CUMP'] / divisor).round(2)
    resultado.drop(columns='PON_CUMP', inplace=True)
    return resultado

cumplimiento_P = calcular_cumplimiento_grupo(tabla_general, 'P', 0.25)
cumplimiento_S = calcular_cumplimiento_grupo(tabla_general, 'S', 0.30)
cumplimiento_C = calcular_cumplimiento_grupo(tabla_general, 'C', 0.20)
cumplimiento_T = calcular_cumplimiento_grupo(tabla_general, 'T', 0.25)

# === Añadir a tabla_general ===
for df_cumpl in [cumplimiento_P, cumplimiento_S, cumplimiento_C, cumplimiento_T]:
    tabla_general = pd.merge(tabla_general, df_cumpl, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')

# === Añadir %_CUMPLIMIENTO_G (global) ===
def calcular_cumplimiento_global(df):
    temp = df[df['VARIABLE'].isin(columnas_eval_1)].copy()
    temp['PON_CUMP'] = temp['PONDERACION'] * temp['%_CUMPLIMIENTO']
    resultado = (
        temp.groupby(['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'])['PON_CUMP'].sum()
        .reset_index()
        .rename(columns={'PON_CUMP': '%_CUMPLIMIENTO_G'})
    )
    return resultado

cumplimiento_global = calcular_cumplimiento_global(tabla_general)
tabla_general = pd.merge(tabla_general, cumplimiento_global, on=['VALOR_DIMENSIÓN', 'FECHA_EVALUACIÓN'], how='left')


# Cumplimiento por Dimensión

In [ ]:
# === Crear columnas MES y AÑO en el DataFrame base ===
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True, errors='coerce')

meses_es = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}
df['MES'] = df['FECHA_EVALUACIÓN'].dt.month.map(meses_es)
df['AÑO'] = df['FECHA_EVALUACIÓN'].dt.year

# === Lista de dimensiones a resumir ===
dimensiones_resumen = ['PRODUCTO', 'SEGMENTO', 'R_NR', 'PERIMETRO']
resumen_globales = []

for dim in dimensiones_resumen:
    # 1️⃣ Calcular cantidad de errores (falsos) por MES, AÑO y dimensión
    errores = (
        df.groupby([dim, 'MES', 'AÑO'])[columnas_eval_1]
        .apply(lambda x: (x == False).sum())
        .reset_index()
    )

    # 2️⃣ Calcular cantidad de llamadas
    llamadas = (
        df.groupby([dim, 'MES', 'AÑO'])
        .size()
        .reset_index(name='NUM_LLAMADAS')
    )

    # 3️⃣ Unir errores con cantidad de llamadas
    temp = pd.merge(errores, llamadas, on=[dim, 'MES', 'AÑO'])

    # 4️⃣ Pasar a formato largo
    temp = temp.melt(
        id_vars=[dim, 'MES', 'AÑO', 'NUM_LLAMADAS'],
        value_vars=columnas_eval_1,
        var_name='VARIABLE',
        value_name='N_FALSOS'
    )

    # 5️⃣ Calcular % cumplimiento y ponderaciones
    temp['%_CUMPLIMIENTO'] = ((1 - temp['N_FALSOS'] / temp['NUM_LLAMADAS']) * 100).round(2)
    temp['PONDERACION'] = temp['VARIABLE'].map(ponderacion_dict)
    temp['PON_CUMP'] = (temp['%_CUMPLIMIENTO'] / 100) * temp['PONDERACION']

    # 6️⃣ Agregar resultado ponderado por MES, AÑO y dimensión
    resumen = (
        temp.groupby([dim, 'MES', 'AÑO'], as_index=False)
        .agg({'PON_CUMP': 'sum', 'NUM_LLAMADAS': 'first'})
    )
    resumen['%_CUMPLIMIENTO_GLOBAL'] = (resumen['PON_CUMP'] * 100).round(2)

    # 7️⃣ Renombrar y agregar columna DIMENSION
    resumen = resumen.rename(columns={dim: 'VALOR_DIMENSION'})
    resumen['DIMENSION'] = dim

    # 8️⃣ Guardar resultado
    resumen_globales.append(
        resumen[['DIMENSION', 'VALOR_DIMENSION', 'MES', 'AÑO', 'NUM_LLAMADAS', '%_CUMPLIMIENTO_GLOBAL']]
    )

# 🔟 Unir todo en una sola tabla llamada valor_dimension
valor_dimension = pd.concat(resumen_globales, ignore_index=True)

# ✅ Mostrar resultado final
print("✅ Tabla consolidada 'valor_dimension' creada:")
valor_dimension.head()

✅ Tabla consolidada 'valor_dimension' creada:


,DIMENSION,VALOR_DIMENSION,MES,AÑO,NUM_LLAMADAS,%_CUMPLIMIENTO_GLOBAL
0,PRODUCTO,HIPOTECARIO,Abril,2025,1,100.00
1,PRODUCTO,MICROBANK,Abril,2025,634,99.69
2,PRODUCTO,PAYMENT,Abril,2025,4661,99.23
3,PRODUCTO,PRESTAMOS,Abril,2025,4658,99.54
4,PRODUCTO,RENTING,Abril,2025,38,100.00


# Cumplimiento Empresa - Dimensión NRN

In [ ]:
llamadas = pd.read_parquet(ruta_historical)
# === Asegurarse de que R_NR existe y está bien formateado ===
if 'R_NR' not in df.columns:
    raise ValueError("⚠️ La columna 'R_NR' no existe en el DataFrame.")

df['R_NR'] = df['R_NR'].astype(str).str.upper().str.strip()

# === Derivar EMPRESA a partir de SERVICIO (si aún no lo hiciste) ===
df['EMPRESA'] = df['SERVICIO'].str.split('_').str[0]

# === Crear columnas de MES y AÑO ===
meses_es = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True, errors='coerce')
df['MES'] = df['FECHA_EVALUACIÓN'].dt.month.map(meses_es)
df['AÑO'] = df['FECHA_EVALUACIÓN'].dt.year

# === Transformar tabla a formato largo y calcular ponderaciones ===
tabla_emp_rnr = df.melt(
    id_vars=['EMPRESA', 'R_NR', 'MES', 'AÑO'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

tabla_emp_rnr['ACIERTO'] = tabla_emp_rnr['ACIERTO'].astype(int)
tabla_emp_rnr['PONDERACION'] = tabla_emp_rnr['VARIABLE'].map(ponderacion_dict)
tabla_emp_rnr['PON_CUMP'] = tabla_emp_rnr['ACIERTO'] * tabla_emp_rnr['PONDERACION']

# === Calcular % cumplimiento ponderado por EMPRESA y R_NR ===
cumplimiento_rnr = (
    tabla_emp_rnr
    .groupby(['EMPRESA', 'R_NR', 'MES', 'AÑO'], as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_rnr['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_rnr['PON_CUMP'] / cumplimiento_rnr['PONDERACION']) * 100
).round(2)

tabla_cumplimiento_rnr = cumplimiento_rnr[['EMPRESA', 'R_NR', 'MES', 'AÑO', '%_CUMPLIMIENTO_GLOBAL']]

# === Mostrar resultado final ===
print("✅ Cumplimiento por EMPRESA y R_NR:")
tabla_cumplimiento_rnr[['EMPRESA', 'R_NR', 'MES', 'AÑO', '%_CUMPLIMIENTO_GLOBAL']].head()


✅ Cumplimiento por EMPRESA y R_NR:


,EMPRESA,R_NR,MES,AÑO,%_CUMPLIMIENTO_GLOBAL
0,ADV,NR,Abril,2025,100.00
1,AXA,NR,Abril,2025,99.43
2,AXA,R,Abril,2025,98.93
3,COL,NR,Abril,2025,99.35
4,COL,R,Abril,2025,99.31


# Perimetro +90 o -90 por fecha de informe y evaluación

In [ ]:
# === Variables y ponderaciones ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6',
                   'S1', 'S2', 'S3', 'S4', 'S5M',
                   'C1M', 'C2M',
                   'T1M', 'T2M', 'T3']

ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10,
    'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Asegurar que FECHA_EVALUACIÓN sea datetime ===
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True)

# === Función para calcular primer día hábil después de la evaluación ===
def siguiente_dia_habil(fecha):
    dia = fecha + pd.Timedelta(days=1)
    while dia.weekday() >= 5:  # 5=sábado, 6=domingo
        dia += pd.Timedelta(days=1)
    return dia

df['FECHA_INFORME'] = df['FECHA_EVALUACIÓN'].apply(siguiente_dia_habil)

# === Preparar tabla para cálculo ===
tabla = df.copy()
tabla['NUM_LLAMADAS'] = 1  # cada fila representa una llamada

tabla_melt = tabla.melt(
    id_vars=['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'SEGMENTO'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

# Transformar a 0/1: True = 1, False = 0
tabla_melt['ACIERTO'] = tabla_melt['ACIERTO'].astype(int)
tabla_melt['PONDERACION'] = tabla_melt['VARIABLE'].map(ponderacion_dict)
tabla_melt['PON_CUMP'] = tabla_melt['ACIERTO'] * tabla_melt['PONDERACION']

# === Filtrar por SEGMENTO 'mas90' y 'menos90' ===
tabla_filtrada = tabla_melt[tabla_melt['SEGMENTO'].isin(['mas90', 'menos90'])].copy()

# === % de cumplimiento ponderado por FECHA_EVALUACIÓN, FECHA_INFORME y SEGMENTO ===
cumplimiento_fecha_dim = (
    tabla_filtrada
    .groupby(['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'SEGMENTO'], as_index=False)
    .agg({'PON_CUMP':'sum', 'PONDERACION':'sum'})
)
cumplimiento_fecha_dim['%_CUMPLIMIENTO_GLOBAL'] = ((cumplimiento_fecha_dim['PON_CUMP'] / cumplimiento_fecha_dim['PONDERACION']) * 100).round(2)
cumplimiento_fecha_dim.rename(columns={'SEGMENTO':'VALOR_DIMENSIÓN'}, inplace=True)

# === Calcular GLOBAL por FECHA_EVALUACIÓN y FECHA_INFORME ===
cumplimiento_global = (
    cumplimiento_fecha_dim
    .groupby(['FECHA_EVALUACIÓN', 'FECHA_INFORME'], as_index=False)
    .agg({'PON_CUMP':'sum', 'PONDERACION':'sum'})
)
cumplimiento_global['%_CUMPLIMIENTO_GLOBAL'] = ((cumplimiento_global['PON_CUMP'] / cumplimiento_global['PONDERACION']) * 100).round(2)
cumplimiento_global['VALOR_DIMENSIÓN'] = 'GLOBAL'

# === Unir ambos resultados ===
tabla_cumplimiento_final = pd.concat([cumplimiento_fecha_dim, cumplimiento_global], ignore_index=True)

# === Ordenar y limpiar columnas finales ===
tabla_cumplimiento_final = tabla_cumplimiento_final.sort_values(['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VALOR_DIMENSIÓN'])
tabla_cumplimiento_final = tabla_cumplimiento_final[['FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VALOR_DIMENSIÓN', '%_CUMPLIMIENTO_GLOBAL']]

tabla_cumplimiento_final.head(20)

,FECHA_EVALUACIÓN,FECHA_INFORME,VALOR_DIMENSIÓN,%_CUMPLIMIENTO_GLOBAL
44,2025-04-01,2025-04-02,GLOBAL,99.44
0,2025-04-01,2025-04-02,mas90,99.80
1,2025-04-01,2025-04-02,menos90,99.06
45,2025-04-02,2025-04-03,GLOBAL,99.42
2,2025-04-02,2025-04-03,mas90,99.76
3,2025-04-02,2025-04-03,menos90,99.07
46,2025-04-03,2025-04-04,GLOBAL,99.62
4,2025-04-03,2025-04-04,mas90,99.86
5,2025-04-03,2025-04-04,menos90,99.41
47,2025-04-04,2025-04-07,GLOBAL,99.32


# Cumplimiento variable, tipo variable y descripción

In [ ]:
# === Agregar columna que identifica el tipo de variable (la primera letra) ===
tabla_melt['TIPO_VARIABLE'] = tabla_melt['VARIABLE'].str[0]

# === Calcular cumplimiento total por tipo (P, S, C, T) ===
cumplimiento_tipo_total = (
    tabla_melt
    .groupby('TIPO_VARIABLE', as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_tipo_total['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_tipo_total['PON_CUMP'] / cumplimiento_tipo_total['PONDERACION']) * 100
).round(2)

# === Calcular cumplimiento individual por variable ===
cumplimiento_variable_total = (
    tabla_melt
    .groupby(['TIPO_VARIABLE', 'VARIABLE'], as_index=False)
    .agg({'PON_CUMP': 'sum', 'PONDERACION': 'sum'})
)
cumplimiento_variable_total['%_CUMPLIMIENTO_GLOBAL'] = (
    (cumplimiento_variable_total['PON_CUMP'] / cumplimiento_variable_total['PONDERACION']) * 100
).round(2)

# === Agregar columna de descripción de la variable ===
mapeo_descripciones = {
    'P1': 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA',
    'P2': 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD',
    'P3': 'ESCUCHA COMPRENSIVA Y EMPÁTICA',
    'P4': 'TRATO CORDIAL AL CLIENTE',
    'P5': 'RESPONSABILIDAD CORPORATIVA',
    'P6': 'DESPEDIDA CORPORATIVA',
    'S1': 'INFORMACIÓN MOTIVO DE LA LLAMADA',
    'S2': 'SITUACIÓN ACTUAL DEL DEUDOR',
    'S3': 'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES',
    'S4': 'DIRECCIÓN',
    'S5M': 'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS',
    'C1M': 'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO',
    'C2M': 'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE',
    'T1M': 'USO EFECTIVO DEL SCRIPT',
    'T2M': 'CAPACIDAD DE EXPRESIÓN',
    'T3': 'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN'
}

cumplimiento_variable_total['DESCRIPCION_VARIABLE'] = cumplimiento_variable_total['VARIABLE'].map(mapeo_descripciones)

# Las filas TOTAL estarán vacías en descripción
cumplimiento_tipo_total['VARIABLE'] = 'TOTAL'
cumplimiento_tipo_total['DESCRIPCION_VARIABLE'] = ''

# === Combinar totales por tipo y variables individuales ===
tabla_categorias = pd.concat([
    cumplimiento_tipo_total[['TIPO_VARIABLE', 'VARIABLE', 'DESCRIPCION_VARIABLE', '%_CUMPLIMIENTO_GLOBAL']],
    cumplimiento_variable_total[['TIPO_VARIABLE', 'VARIABLE', 'DESCRIPCION_VARIABLE', '%_CUMPLIMIENTO_GLOBAL']]
], ignore_index=True)

# === Ordenar por tipo y variable ===
tabla_categorias = tabla_categorias.sort_values(['TIPO_VARIABLE', 'VARIABLE']).reset_index(drop=True)

# Mostrar resultado final
tabla_categorias.head()



,TIPO_VARIABLE,VARIABLE,DESCRIPCION_VARIABLE,%_CUMPLIMIENTO_GLOBAL
0,C,C1M,OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO,99.68
1,C,C2M,PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS ...,99.70
2,C,TOTAL,,99.69
3,P,P1,"PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN D...",99.41
4,P,P2,IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD,99.68


# Cumplimiento por empresa y variable, con columna ponderación

In [ ]:
# === Diccionario de ponderaciones ===
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10,
    'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Variables de evaluación ===
columnas_eval_1 = list(ponderacion_dict.keys())

# === Agregar columna de descripción de la variable ===
mapeo_descripciones = {
    'P1': 'PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN DE LA LLAMADA',
    'P2': 'IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD',
    'P3': 'ESCUCHA COMPRENSIVA Y EMPÁTICA',
    'P4': 'TRATO CORDIAL AL CLIENTE',
    'P5': 'RESPONSABILIDAD CORPORATIVA',
    'P6': 'DESPEDIDA CORPORATIVA',
    'S1': 'INFORMACIÓN MOTIVO DE LA LLAMADA',
    'S2': 'SITUACIÓN ACTUAL DEL DEUDOR',
    'S3': 'SONDEO: IDENTIFICAR CAPACIDAD DE PAGO Y DETECCIÓN DE NECESIDADES',
    'S4': 'DIRECCIÓN',
    'S5M': 'PREPARACIÓN CIERRE : RESUMEN DE ACUERDOS',
    'C1M': 'OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO',
    'C2M': 'PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS PARA REALIZAR LOS INGRESOS LO ANTES POSIBLE',
    'T1M': 'USO EFECTIVO DEL SCRIPT',
    'T2M': 'CAPACIDAD DE EXPRESIÓN',
    'T3': 'BUSCAR EL COMPROMISO DE PAGO ENUNCIANDO LOS BENEFICIOS Y LAS CONSECUENCIAS DE LA REGULARIZACIÓN'
}

# === Supongo que df ya está cargado y tiene las columnas EMPRESA y las variables ===

# Convertir ACIERTO a int y hacer melt para tener variables en fila
tabla_melt = df.melt(
    id_vars=['EMPRESA'],  # asegúrate que está EMPRESA, agrega más columnas si quieres
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

tabla_melt['ACIERTO'] = tabla_melt['ACIERTO'].astype(int)

# Mapear ponderación y descripción
tabla_melt['PONDERACION'] = tabla_melt['VARIABLE'].map(ponderacion_dict)
tabla_melt['DESCRIPCION_VARIABLE'] = tabla_melt['VARIABLE'].map(mapeo_descripciones)

# Calcular % cumplimiento simple (promedio) por EMPRESA y VARIABLE
cumplimiento_variable_empresa = (
    tabla_melt
    .groupby(['EMPRESA', 'VARIABLE', 'DESCRIPCION_VARIABLE', 'PONDERACION'], as_index=False)
    .agg({'ACIERTO': 'mean'})
)

# Pasar a porcentaje y redondear
cumplimiento_variable_empresa['%_CUMPLIMIENTO_GLOBAL'] = (cumplimiento_variable_empresa['ACIERTO'] * 100).round(2)

# Seleccionar columnas finales sin ponderar el % cumplimiento
cumplimiento_variable_empresa = cumplimiento_variable_empresa[
    ['EMPRESA', 'VARIABLE', 'DESCRIPCION_VARIABLE', 'PONDERACION', '%_CUMPLIMIENTO_GLOBAL']
]

# Mostrar resultado
print("✅ Cumplimiento simple por variable y empresa (con ponderación y descripción):")
display(cumplimiento_variable_empresa.head(10))



✅ Cumplimiento simple por variable y empresa (con ponderación y descripción):


,EMPRESA,VARIABLE,DESCRIPCION_VARIABLE,PONDERACION,%_CUMPLIMIENTO_GLOBAL
0,ADV,C1M,OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO,0.10,100.0
1,ADV,C2M,PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS ...,0.10,100.0
2,ADV,P1,"PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN D...",0.02,100.0
3,ADV,P2,IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD,0.10,100.0
4,ADV,P3,ESCUCHA COMPRENSIVA Y EMPÁTICA,0.03,100.0
5,ADV,P4,TRATO CORDIAL AL CLIENTE,0.03,100.0
6,ADV,P5,RESPONSABILIDAD CORPORATIVA,0.05,100.0
7,ADV,P6,DESPEDIDA CORPORATIVA,0.02,100.0
8,ADV,S1,INFORMACIÓN MOTIVO DE LA LLAMADA,0.03,100.0
9,ADV,S2,SITUACIÓN ACTUAL DEL DEUDOR,0.02,100.0


**Cumplimiento por ID, fecha evaluación y  variable**

In [ ]:
#Crear Tabla Variables por ID y Fecha_Evaluación

# 1️⃣ Leer archivo histórico
df_hist = pd.read_parquet(ruta_historical)

# 2️⃣ Asegurar que FECHA_EVALUACIÓN sea datetime
df_hist['FECHA_EVALUACIÓN'] = pd.to_datetime(df_hist['FECHA_EVALUACIÓN'], dayfirst=True, errors='coerce')

# 3️⃣ Lista de variables y ponderaciones
columnas_eval = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6',
                 'S1', 'S2', 'S3', 'S4', 'S5M', 'C1M', 'C2M', 'T1M', 'T2M', 'T3']
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# 4️⃣ Pasar a formato largo
tabla_cumplimiento_variables = df_hist.melt(
    id_vars=['ID', 'FECHA_EVALUACIÓN'],
    value_vars=columnas_eval,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

# 5️⃣ Calcular % de cumplimiento como en tu código original
# True = cumple, False = no cumple
# En este caso, 1 llamado por ID → %_CUMPLIMIENTO = 100 si cumple, 0 si no
tabla_cumplimiento_variables['%_CUMPLIMIENTO'] = (1 - (~tabla_cumplimiento_variables['ACIERTO'].astype(bool))) * 100

# 6️⃣ Seleccionar solo columnas finales
tabla_cumplimiento_variables = tabla_cumplimiento_variables[['ID', 'FECHA_EVALUACIÓN', 'VARIABLE', '%_CUMPLIMIENTO']]

# 7️⃣ Mostrar resultado
tabla_cumplimiento_variables.head()



,ID,FECHA_EVALUACIÓN,VARIABLE,%_CUMPLIMIENTO
0,1245S_152-OP_1195_17_20250326H_13_07,2025-04-01,P1,100
1,1245S_191-OP_2050_497_20250226H_19_04,2025-04-01,P1,100
2,1245S_195-OP_1698_19_20250328H_13_38,2025-04-01,P1,100
3,1245S_195-OP_890_19_20250328H_14_47,2025-04-01,P1,100
4,1245S_53-OP_1231_163_20250327H_11_56,2025-04-01,P1,100


# Cumplimiento +90, -90 y global por empresa y fecha de evaluación

In [ ]:
# === Derivar EMPRESA a partir de SERVICIO ===
df['EMPRESA'] = df['SERVICIO'].str.split('_').str[0]

# Asegurar que FECHA_EVALUACIÓN es datetime
df['FECHA_EVALUACIÓN'] = pd.to_datetime(df['FECHA_EVALUACIÓN'], dayfirst=True, errors='coerce')

# === Preparar tabla base con EMPRESA, SEGMENTO, FECHA_EVALUACIÓN ===
tabla_base = df.melt(
    id_vars=['EMPRESA', 'SEGMENTO', 'FECHA_EVALUACIÓN'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='ACIERTO'
)

# Transformar a 0/1 y agregar ponderación y descripción
tabla_base['ACIERTO'] = tabla_base['ACIERTO'].astype(int)
tabla_base['PONDERACION'] = tabla_base['VARIABLE'].map(ponderacion_dict)
tabla_base['DESCRIPCION_VARIABLE'] = tabla_base['VARIABLE'].map(mapeo_descripciones)

# === Calcular % cumplimiento por EMPRESA, SEGMENTO y FECHA_EVALUACIÓN ===
cumplimiento_empresa_seg_fecha = (
    tabla_base
    .groupby(['EMPRESA', 'SEGMENTO', 'FECHA_EVALUACIÓN'], as_index=False)
    .agg({'ACIERTO': 'mean'})
)
cumplimiento_empresa_seg_fecha['%_CUMPLIMIENTO'] = (cumplimiento_empresa_seg_fecha['ACIERTO'] * 100).round(2)

# === Pivotear por segmento ===
tabla_empresa_fecha = cumplimiento_empresa_seg_fecha.pivot_table(
    index=['EMPRESA', 'FECHA_EVALUACIÓN'],
    columns='SEGMENTO',
    values='%_CUMPLIMIENTO'
).reset_index()

# === Calcular cumplimiento GLOBAL (promedio de ambos segmentos) ===
tabla_empresa_fecha['%_CUMPLIMIENTO_GLOBAL'] = (
    tabla_empresa_fecha[['mas90', 'menos90']].mean(axis=1)
).round(2)

# === Renombrar columnas ===
tabla_empresa_fecha.rename(columns={
    'menos90': '%_CUMPLIMIENTO_0_90',
    'mas90': '%_CUMPLIMIENTO_MAS_90'
}, inplace=True)

# === Mostrar resultado final ===
print("✅ Cumplimiento por empresa, fecha y global:")
display(tabla_empresa_fecha.head())



✅ Cumplimiento por empresa, fecha y global:


SEGMENTO,EMPRESA,FECHA_EVALUACIÓN,%_CUMPLIMIENTO_MAS_90,%_CUMPLIMIENTO_0_90,%_CUMPLIMIENTO_GLOBAL
0,ADV,2025-04-01,100.0,NaN,100.00
1,ADV,2025-04-08,100.0,NaN,100.00
2,AXA,2025-04-01,100.0,98.29,99.15
3,AXA,2025-04-02,100.0,97.85,98.92
4,AXA,2025-04-03,100.0,99.19,99.60


# Cumplimiento por empresa, variable y fecha evalaución

In [ ]:
# === 1️⃣ Diccionario de ponderaciones ===
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === 2️⃣ Añadir ponderación y tipo de variable ===
tabla_emp['PONDERACION'] = tabla_emp['VARIABLE'].map(ponderacion_dict)
tabla_emp['TIPO_VARIABLE'] = tabla_emp['VARIABLE'].str[0]

# === 3️⃣ Calcular % de cumplimiento promedio por empresa y variable ===
cumplimiento_variable_empresa = (
    tabla_emp
    .groupby(['EMPRESA', 'TIPO_VARIABLE', 'VARIABLE'], as_index=False)
    .agg({'ACIERTO': 'mean'})
)

# === 4️⃣ Calcular % de cumplimiento final ===
cumplimiento_variable_empresa['%_CUMPLIMIENTO'] = (cumplimiento_variable_empresa['ACIERTO'] * 100).round(2)

# === 5️⃣ Agregar descripción y ponderación ===
cumplimiento_variable_empresa['DESCRIPCION_VARIABLE'] = cumplimiento_variable_empresa['VARIABLE'].map(mapeo_descripciones)
cumplimiento_variable_empresa['PONDERACION'] = cumplimiento_variable_empresa['VARIABLE'].map(ponderacion_dict)

# === 6️⃣ Reordenar columnas ===
cumplimiento_variable_empresa = cumplimiento_variable_empresa[
    ['EMPRESA', 'TIPO_VARIABLE', 'VARIABLE', 'DESCRIPCION_VARIABLE', 'PONDERACION', '%_CUMPLIMIENTO']
]

# === 7️⃣ Mostrar resultado final ===
print("✅ Tabla final de cumplimiento por empresa y variable (ponderada correctamente)")
display(cumplimiento_variable_empresa.head(20))


✅ Tabla final de cumplimiento por empresa y variable (ponderada correctamente)


,EMPRESA,TIPO_VARIABLE,VARIABLE,DESCRIPCION_VARIABLE,PONDERACION,%_CUMPLIMIENTO
0,ADV,C,C1M,OFRECER TPV COMO ALTERNATIVA PRIORITARIA DE PAGO,0.10,100.00
1,ADV,C,C2M,PRIORIZA LOS CANALES DIGITALES U OTROS MEDIOS ...,0.10,100.00
2,ADV,P,P1,"PRESENTACIÓN, SALUDO CORPORATIVO Y GRABACIÓN D...",0.02,100.00
3,ADV,P,P2,IDENTIFICACIÓN DEL INTERLOCUTOR - LOPD,0.10,100.00
4,ADV,P,P3,ESCUCHA COMPRENSIVA Y EMPÁTICA,0.03,100.00
5,ADV,P,P4,TRATO CORDIAL AL CLIENTE,0.03,100.00
6,ADV,P,P5,RESPONSABILIDAD CORPORATIVA,0.05,100.00
7,ADV,P,P6,DESPEDIDA CORPORATIVA,0.02,100.00
8,ADV,S,S1,INFORMACIÓN MOTIVO DE LA LLAMADA,0.03,100.00
9,ADV,S,S2,SITUACIÓN ACTUAL DEL DEUDOR,0.02,100.00


# Editar la BBDD para generar tabla de hechos en power bi

In [ ]:
#Expandir la columna servicio de BBDD

ruta_bbdd = '/content/drive/MyDrive/COPS/BBDD.xlsx'

# 1️⃣ Leer el Excel usando la ruta ya definida
bbdd = pd.read_excel(ruta_bbdd)

# 2️⃣ Revisar que la columna SERVICIO exista
if 'SERVICIO' not in bbdd.columns:
    raise ValueError("La columna 'SERVICIO' no existe en el archivo.")

# 3️⃣ Dividir la columna SERVICIO en 5 partes
nuevas_columnas = ['EMPRESA', 'PRODUCTO', 'R_NR', 'PERIMETRO', 'SEGMENTO']
bbdd[nuevas_columnas] = bbdd['SERVICIO'].str.split('_', expand=True)

#Eliminar columna servicio
bbdd.drop(columns=["SERVICIO"], inplace=True)

# Guardar nuevamente en Excel (puedes cambiar el nombre de salida)
ruta_salida_bbdd = '/content/drive/MyDrive/COPS/OUTPUT/BBDD_SERVICIO.xlsx'
bbdd.to_excel(ruta_salida_bbdd, index=False)

print("✅ Archivo modificado guardado en:", ruta_salida_bbdd)
print(bbdd["EMPRESA"])



✅ Archivo modificado guardado en: /content/drive/MyDrive/COPS/OUTPUT/BBDD_SERVICIO.xlsx
0       AXA
1       DXC
2       DXC
3       DXC
4       COL
       ... 
9987    COL
9988    COL
9989    COL
9990    COL
9991    COL
Name: EMPRESA, Length: 9992, dtype: object


# Descarga de excel

In [ ]:
# === Exportar a Excel junto con otras hojas ===
with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:

    # 1️⃣ Hojas detalladas por dimensión
    for dim in dimensiones:
        df_dim = tabla_general[tabla_general['DIMENSIÓN'] == dim.capitalize()]
        df_dim.to_excel(writer, sheet_name=dim.capitalize(), index=False)

    # 2️⃣ Tabla general concatenada
    tabla_general.to_excel(writer, sheet_name='TABLA_GENERALCATEG', index=False)

    # 3️⃣ Resumen global por dimensión (versión lista)
    for df_resumen in resumen_globales:
        dim = df_resumen['DIMENSION'].iloc[0]
        df_resumen.to_excel(writer, sheet_name=f'{dim.capitalize()}_GLOBAL', index=False)

    # 4️⃣ Cumplimiento mas90 / menos90 + global por fecha
    tabla_empresa_fecha.to_excel(writer, sheet_name='Cumplimiento_fecha_empresa', index=False)

    # 5️⃣ Tabla categorías con detalle y totales por tipo
    tabla_categorias.to_excel(writer, sheet_name='TABLA_CATEGORIAS', index=False)

    # 5️⃣a️⃣ Añadir tabla_cumplimiento_variables en una hoja nueva
    tabla_cumplimiento_variables.to_excel(writer, sheet_name='CUMPLIMIENTO_VARIABLES', index=False)

    # 6️⃣ Cumplimiento por EMPRESA
    tabla_cumplimiento_empresa.to_excel(writer, sheet_name='CUMPLIMIENTO_EMPRESA', index=False)

    # 7️⃣ Cumplimiento por EMPRESA y R_NR (Calidad Trimestral)
    tabla_cumplimiento_rnr.to_excel(writer, sheet_name='Calidad_Trimestral', index=False)

    # 8️⃣ Cumplimiento por VARIABLE y EMPRESA
    cumplimiento_variable_empresa.to_excel(writer, sheet_name='Calidad_variable_empresa', index=False)

    #Tabla cumplimiento por segmento (+90 y -90)
    tabla_cumplimiento_final.to_excel(writer, sheet_name='Cumplimiento_90', index=False)

print(f"✅ Excel generado correctamente en: {ruta_salida}")




✅ Excel generado correctamente en: /content/drive/MyDrive/COPS/OUTPUT/TABLA_CATEGORIAS.xlsx
